# Implementation of a KANBoost
## Initialisations



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pykan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 7.2 MB/s eta 0:00:00


# Sample inputs

> Add blockquote



In [ ]:

import torch
from sklearn.model_selection import train_test_split

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print("****** DEVICE: ",device , " ******\n")

****** DEVICE:  cuda  ******



# KANBoost Preprocessor Class Implementation

In [ ]:
class Preprocessor:
    def __init__(self, page_size, block_size):
        self.page_size = page_size
        self.block_size = block_size

    def ensure_48bit_address(self, load_address):
        # Ensure the load_address is a 48-bit binary string
        return bin(int(load_address, 16))[2:].zfill(48)

    def calculate_delta(self, block1, block2):
        # Calculate the delta between two blocks (binary subtraction)
        return int(block1, 2) - int(block2, 2)

    def split_load_address(self, load_address):
        binary_address = self.ensure_48bit_address(load_address)

        page = binary_address[:self.page_size]  # x Bit (Varies)
        block = binary_address[self.page_size:self.page_size + self.block_size]  # 6 Bit Fixed
        block_offset = binary_address[self.page_size + self.block_size:]  # Remaining bits

        return (page, block, block_offset)


    def unsplit_load_address(self, load_address, block_delta):
        # Convert the load address (hex) to binary
        binary_address = self.ensure_48bit_address(load_address)

        # Split the binary address into page, block, and offset
        current_page = binary_address[:self.page_size]
        current_block = binary_address[self.page_size:self.page_size + self.block_size]
        current_block_offset = binary_address[self.page_size + self.block_size:]
        adjusted_block_int=int(current_block, 2) + block_delta
        if(adjusted_block_int<0):
          adjusted_block_int=0
        # Adjust the block by adding the block delta
        adjusted_block = bin(adjusted_block_int)[2:].zfill(self.block_size)  # Add delta to block and ensure correct bit length
        # Reconstruct the binary address
        reconstructed_binary_address = current_page + adjusted_block + current_block_offset

        # Convert the reconstructed binary address to hexadecimal
        reconstructed_load_address = hex(int(reconstructed_binary_address, 2))[2:].lower()  # Remove '0x' prefix and convert to uppercase

        return reconstructed_load_address

    def preprocess_data(self, data):
        input_features = []
        output_labels = []
        page_blocks = {}
        preprocessed_details = []  # Store detailed information

        for i in range(len(data) - 1):
            instr_id, cycle_count, load_address, instr_ptr, llc_hit_miss = data[i]
            current_page, current_block, current_block_offset = self.split_load_address(load_address)

            _, _, next_load_address, _, _ = data[i+1]
            next_page, next_block, next_block_offset = self.split_load_address(next_load_address)
            # Initialize page_blocks if current_page is not present
            if current_page not in page_blocks:
                page_blocks[current_page] = ['000001']

            # Calculate delta values for the past blocks
            delta1 = delta2 = delta3 = delta4 = delta5 = 1 + 64

            if len(page_blocks[current_page]) > 1:
                delta1 = 64 + self.calculate_delta(page_blocks[current_page][-1], page_blocks[current_page][-2])
            if len(page_blocks[current_page]) > 2:
                delta2 = 64 + self.calculate_delta(page_blocks[current_page][-2], page_blocks[current_page][-3])
            if len(page_blocks[current_page]) > 3:
                delta3 = 64 + self.calculate_delta(page_blocks[current_page][-3], page_blocks[current_page][-4])
            if len(page_blocks[current_page]) > 4:
                delta4 = 64 + self.calculate_delta(page_blocks[current_page][-4], page_blocks[current_page][-5])
            if len(page_blocks[current_page]) > 5:
                delta5 = 64 + self.calculate_delta(page_blocks[current_page][-4], page_blocks[current_page][-5])

            # Calculate delta for the next block (relative to current block)
            next_delta = self.calculate_delta(next_block, current_block)

            # Append input features
            input_features.append((instr_id,load_address, delta1, delta2, delta3, delta4, delta5))

            # Convert next_delta to a 128-dimensional one-hot array and append as the output label
            output_labels.append(next_delta+64)

            # Append the current block to the page's block list
            page_blocks[current_page].append(current_block)

            # Store the details for inspection without delta1, delta2, delta3, and next_delta
            details = {
                "instr_id": instr_id,
                "page": current_page,
                "block": current_block,
                "block_offset": current_block_offset
            }
            preprocessed_details.append(details)

        return input_features, output_labels, preprocessed_details

# Read Data from file Implementation

In [ ]:
# Reading the data from the text file (same as before)
def read_data_from_file(filename):
    data = []
    with open(filename, 'r', newline='', encoding='utf-8') as file:  # Specify newline='' to handle any line endings
        for line in file:
            line = line.rstrip('\n')  # Strip only the trailing newline to preserve leading newlines if necessary
            if line:
                # Split by comma and remove any extra spaces
                fields = [x.strip() for x in line.split(',')]
                instr_id = int(fields[0])
                cycle_count = int(fields[1])
                load_address = fields[2]
                instr_ptr = fields[3]
                llc_hit_miss = int(fields[4])

                # Append as a tuple
                data.append((instr_id, cycle_count, load_address, instr_ptr, llc_hit_miss))
    return data


# Prepare Dataset Function Implementation

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def prepare_dataset(input_features, output_labels, batch_size=1, device='cpu'):
    """
    Prepares the dataset for training and testing without train-test split, and preserves instruction IDs.

    Args:
        input_features (list or np.array): The input features for the dataset.
        output_labels (list or np.array): The output labels for the dataset.
        batch_size (int): The batch size for data loaders. Default is 1.
        device (str): The device to store the tensors ('cpu' or 'cuda'). Default is 'cpu'.

    Returns:
        dict: A dictionary containing the processed dataset with 'input', 'labels', and 'instr_ids'.
    """
    # Assuming input_features is a list of tuples like (instr_id, feature1, feature2, ..., featureN)
    instr_ids = [x[0] for x in input_features]  # Extracting the instruction IDs
    features = [x[2:] for x in input_features]   # Extracting the actual feature data (excluding instr_id)

    # Convert to PyTorch tensors and move to the specified device
    data_tensor = torch.tensor(features, dtype=torch.float32, device=device)
    target_tensor = torch.tensor(output_labels, dtype=torch.long, device=device)  # Assuming labels are integer values

    # Create data loaders (optional, if you want to batch and shuffle the data)
    data_loader = DataLoader(TensorDataset(data_tensor, target_tensor),
                             batch_size=batch_size, shuffle=True)

    # Initialize tensors for inputs and labels
    all_inputs = torch.empty(0, data_tensor.size(1), device=device)  # Assuming data_tensor has N features
    all_labels = torch.empty(0, dtype=torch.long, device=device)

    # Concatenate all data into a single tensor on the specified device
    for data, labels in data_loader:
        all_inputs = torch.cat((all_inputs, data.to(device)), dim=0)
        all_labels = torch.cat((all_labels, labels.to(device)), dim=0)

    # Return the dataset as a dictionary, including instruction IDs
    dataset = {
        'input': all_inputs,
        'label': all_labels,
        'instr_ids': instr_ids  # Include the instruction IDs in the output (no need to move to device)
    }

    return dataset


  # ------------------------****** Load Dataset for Training only ******  ------------------------
def load_dataset(data,target):
    # Convert to PyTorch tensors
   #This needs to be torch.float32

    # Split dataset into train and test sets
    data_tensor = torch.tensor(data, dtype=torch.float32)
    target_tensor = torch.tensor(target, dtype=torch.long)

    # Split dataset deterministically (e.g., first 80% for training, last 20% for testing)
    split_index = int(len(data_tensor) * 0.8)  # 80-20 split

    train_data = data_tensor[:split_index]
    test_data = data_tensor[split_index:]
    train_target = target_tensor[:split_index]
    test_target = target_tensor[split_index:]

    # Create data loaders (optional, if you want to batch and shuffle the data)
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_target), batch_size=1, shuffle=False)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_target), batch_size=1, shuffle=False)

    train_inputs = torch.empty(0, 5, device=device)
    train_labels = torch.empty(0, dtype=torch.long,device=device)
    test_inputs = torch.empty(0, 5, device=device)
    test_labels = torch.empty(0,dtype=torch.long,  device=device)

    # Concatenate all data into a single tensor on the specified device
    for data, labels in train_loader:
        train_inputs = torch.cat((train_inputs, data.to(device)), dim=0)
        train_labels = torch.cat((train_labels, labels.to(device)), dim=0)

    for data, labels in test_loader:
        test_inputs = torch.cat((test_inputs, data.to(device)), dim=0)
        test_labels = torch.cat((test_labels, labels.to(device)), dim=0)

    dataset = {}
    dataset['train_input'] = train_inputs
    dataset['test_input'] = test_inputs
    dataset['train_label'] = train_labels
    dataset['test_label'] = test_labels

    return dataset

# KANBoost Entry Point:

In [60]:
filename = '/content/drive/MyDrive/482.sphinx3-s2.trace.gz-hashed_perceptron-no-no-no-trace-lru-1core.txt'
data = read_data_from_file(filename)

# Initialize the KANBoost Preprocessor
page_size = 36
block_size = 6

preprocessor = Preprocessor(page_size, block_size)

# Use the preprocess_data function to process the entire data and capture details
input_features, output_labels, preprocessed_details = preprocessor.preprocess_data(data)
# dataset=prepare_dataset(input_features=input_features,output_labels=output_labels,device=device)

In [61]:
input_features[0]

(3, '5cb41dc41440', 65, 65, 65, 65, 65)

In [72]:
last_5_elements = [t[-5:] for t in input_features]

In [73]:
dataset=load_dataset(last_5_elements,output_labels)

In [ ]:
print("Train data shape: {}".format(dataset['train_input'].shape))
print("Train target shape: {}".format(dataset['train_label'].shape))
print("Test data shape: {}".format(dataset['test_input'].shape))
print("Test target shape: {}".format(dataset['test_label'].shape))


Train data shape: torch.Size([234214, 5])
Train target shape: torch.Size([234214])
Test data shape: torch.Size([58554, 5])
Test target shape: torch.Size([58554])


## Creating and Training the Echo state network

*   List item
*   List item



In [66]:
class DeepEchoStateNetwork(torch.nn.Module):
    def __init__(self, input_dim, reservoir_sizes, output_dim, spectral_radius=0.95):
        super(DeepEchoStateNetwork, self).__init__()
        self.reservoir_sizes = reservoir_sizes

        # Initialize weight matrices for each reservoir layer
        self.Win_layers = []
        self.W_layers = []

        for i, reservoir_size in enumerate(reservoir_sizes):
            prev_dim = input_dim if i == 0 else reservoir_sizes[i - 1]

            # Input-to-reservoir weights
            Win = torch.randn(reservoir_size, prev_dim) * 0.1
            self.Win_layers.append(Win)

            # Reservoir-to-reservoir weights
            W = torch.randn(reservoir_size, reservoir_size) * 0.1
            max_eigenvalue = max(abs(torch.linalg.eigvals(W).real))
            W_scaled = W * (spectral_radius / max_eigenvalue)
            self.W_layers.append(W_scaled)

        # Register the weights as buffers (non-trainable parameters)
        for i, (Win, W) in enumerate(zip(self.Win_layers, self.W_layers)):
            self.register_buffer(f'Win_{i}', Win)
            self.register_buffer(f'W_{i}', W)

        # Output layer (readout)
        self.Wout = torch.nn.Linear(reservoir_sizes[-1], output_dim)

    def forward(self, x):
        device = x.device
        batch_size = x.shape[0]

        # Initialize reservoir state
        reservoir_state = None

        # Iterate through reservoir layers
        for i in range(len(self.reservoir_sizes)):
            # Get weights for this layer
            Win = getattr(self, f'Win_{i}').to(device)
            W = getattr(self, f'W_{i}').to(device)

            if reservoir_state is None:
                reservoir_state = torch.zeros(batch_size, Win.shape[0], device=device)

            # Update reservoir state
            reservoir_state = torch.tanh(
                torch.matmul(x, Win.T) + torch.matmul(reservoir_state, W.T)
            )

            # Pass state to the next layer
            x = reservoir_state

        # Output layer
        return self.Wout(reservoir_state)

In [67]:
def clear_gpu_memory():
    """Clear GPU memory cache."""
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()  # Wait for all streams on a CUDA device to finish
    print("GPU memory cleared.")
clear_gpu_memory()

GPU memory cleared.


In [74]:
def Train_KANBoost(dataset, model, optimizer, criterion):
    def train_acc():
        with torch.no_grad():
            predictions = torch.argmax(model(dataset['train_input']), dim=1)
            return torch.mean((predictions == dataset['train_label']).float())

    def test_acc():
        with torch.no_grad():
            predictions = torch.argmax(model(dataset['test_input']), dim=1)
            return torch.mean((predictions == dataset['test_label']).float())

    # Training step
    optimizer.zero_grad()
    outputs = model(dataset['train_input'])
    loss = criterion(outputs, dataset['train_label'])
    loss.backward()
    optimizer.step()

    # Evaluate metrics
    train_accuracy = train_acc()
    test_accuracy = test_acc()

    print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, Loss: {loss.item():.4f}")

In [75]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def train_model(dataset, num_epochs=10, input_dim=None, output_dim=None, reservoir_sizes=[100]):
    # Determine the device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move input and labels to the appropriate device (not the TensorDataset itself)
    train_input = dataset['train_input'].to(device)
    train_label = dataset['train_label'].to(device)
    test_input = dataset['test_input'].to(device)
    test_label = dataset['test_label'].to(device)

    # Step 1: Create TensorDataset for train and test datasets
    train_dataset = TensorDataset(train_input, train_label)
    test_dataset = TensorDataset(test_input, test_label)

    # Step 2: Create DataLoader for both train and test datasets
    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize Deep ESN model
    esn_model = DeepEchoStateNetwork(input_dim, reservoir_sizes, output_dim).to(device)  # Move model to device
    optimizer = torch.optim.Adam(esn_model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    # Step 3: Iterate through the DataLoader for training
    for epoch in range(num_epochs):
        print("Epoch going",epoch)
        for train_batch, test_batch in zip(train_loader, test_loader):
            train_inputs, train_targets = train_batch
            test_inputs, test_targets = test_batch

            # Move batch data to the same device
            train_inputs = train_inputs.to(device)
            train_targets = train_targets.to(device)
            test_inputs = test_inputs.to(device)
            test_targets = test_targets.to(device)

            partial_dataset = {
                "train_input": train_inputs,
                "train_label": train_targets,
                "test_input": test_inputs,
                "test_label": test_targets
            }
            Train_KANBoost(partial_dataset, esn_model, optimizer, criterion)

    # Save the trained model
    torch.save(esn_model.state_dict(), "trained_esn_model.pth")

In [76]:
input_dim = dataset['train_input'].shape[1]
indivsize=1000
num_layers=6
reservoir_sizes=[]
for i in range(0,num_layers):
  reservoir_sizes.append(num_layers)
train_model(dataset, num_epochs=50, input_dim=input_dim, output_dim=128, reservoir_sizes=reservoir_sizes)
print("Model saved to trained_esn_model.pth")

Streaming output truncated to the last 5000 lines.
Train Accuracy: 0.2656, Test Accuracy: 0.4688, Loss: 3.5452
Train Accuracy: 0.2500, Test Accuracy: 0.3906, Loss: 2.9456
Train Accuracy: 0.2812, Test Accuracy: 0.4375, Loss: 2.7161
Train Accuracy: 0.3438, Test Accuracy: 0.4688, Loss: 2.8517
Train Accuracy: 0.3438, Test Accuracy: 0.3750, Loss: 3.1246
Train Accuracy: 0.3906, Test Accuracy: 0.4531, Loss: 3.0976
Train Accuracy: 0.1875, Test Accuracy: 0.4062, Loss: 3.4871
Train Accuracy: 0.2500, Test Accuracy: 0.4062, Loss: 3.3976
Train Accuracy: 0.3125, Test Accuracy: 0.3906, Loss: 3.1339
Train Accuracy: 0.2344, Test Accuracy: 0.4375, Loss: 3.5875
Train Accuracy: 0.2031, Test Accuracy: 0.4375, Loss: 3.4315
Train Accuracy: 0.3281, Test Accuracy: 0.4062, Loss: 3.2347
Train Accuracy: 0.3281, Test Accuracy: 0.4531, Loss: 3.0403
Train Accuracy: 0.1875, Test Accuracy: 0.3750, Loss: 3.5196
Train Accuracy: 0.1875, Test Accuracy: 0.4688, Loss: 3.1448
Train Accuracy: 0.2812, Test Accuracy: 0.4219, Lo

In [ ]:
cp -r /content/model /content/drive/MyDrive/kanboost3_24

# Prefetch File Generation


In [77]:
dataset=prepare_dataset(input_features=input_features,output_labels=output_labels,device=device)

In [ ]:
dataset['input'][0:5]

tensor([[65., 65., 65., 65., 65.],
        [65., 65., 65., 65., 65.],
        [65., 65., 65., 65., 65.],
        [65., 76., 66., 65., 65.],
        [65., 65., 65., 15., 15.]], device='cuda:0')

In [78]:
output_dim=128
indivsize=1000
num_layers=6
reservoir_sizes=[]
for i in range(0,num_layers):
  reservoir_sizes.append(num_layers)
esn_model = DeepEchoStateNetwork(input_dim, reservoir_sizes, output_dim,spectral_radius=0.95)

# Load the saved state
esn_model.load_state_dict(torch.load("trained_esn_model.pth"))

# Move to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
esn_model = esn_model.to(device)

# Set the model to evaluation mode
esn_model.eval()



<ipython-input-78-7b355e755d68>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  esn_model.load_state_dict(torch.load("trained_esn_model.pth"))


DeepEchoStateNetwork(
  (Wout): Linear(in_features=6, out_features=128, bias=True)
)

In [ ]:
!ls

0.7_cache_data.zip  0.7_state	   drive	kanboost  sample_data
0.7_config.yml	    0.7_state.zip  history.txt	model


In [ ]:
!cd "model"
!ls

drive  model  sample_data


In [79]:

def generate_prefetch_file(path, prefetches):
    with open(path, 'w') as f:
        for instr_id, pf_addr in prefetches:
            print(instr_id, pf_addr, file=f)
def prefetch_generation(batch_size=5000):
    """
    Prepare dataset with manual batching.
    Returns the average inference time per sample in nanoseconds.
    """
    import time

    # Get the total number of samples
    total_samples = len(input_features)
    total_inference_time = 0
    num_batches = 0
    prefetches = []

    # Create batches manually by splitting the data
    for i in range(0, total_samples, batch_size):
        batch_inputs = dataset['input'][i:i + batch_size]
        num_batches += 1
        batch_size_actual = len(batch_inputs)  # Adjust for the last batch

        # Measure inference time
        start_time = time.time()
        pred = torch.argmax(esn_model(batch_inputs), dim=1)
        inference_time = time.time() - start_time

        # Accumulate total inference time
        total_inference_time += inference_time

        # Process predictions
        for j in range(len(pred)):
            instr_id, load_address, _, _, _, _, _ = input_features[i + j]
            load_addr = preprocessor.unsplit_load_address(load_address, int(pred[j].item()) - 64)
            prefetches.append((instr_id, load_addr))

    # Calculate average inference time per sample in nanoseconds
    avg_inference_time_per_sample_ns = (total_inference_time / total_samples) * 1e9 if total_samples > 0 else 0
    print(f"Average Inference Time per Sample: {avg_inference_time_per_sample_ns:.2f} nanoseconds")

    return avg_inference_time_per_sample_ns, prefetches


In [80]:
prefetches=[]
avgtime,prefetches=prefetch_generation()
generate_prefetch_file('prefetch_1M_model_v0.5.txt', prefetches)

Average Inference Time per Sample: 129.75 nanoseconds


In [ ]:
len(prefetches),len(input_features)

(260492, 260492)